In [1]:
from detectron2.config import get_cfg
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import MetadataCatalog, DatasetCatalog
import detectron2.data.transforms as T
from detectron2.structures.image_list import ImageList
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.modeling.box_regression import Box2BoxTransform
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetMapper
from detectron2.data import (
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.data.samplers import TrainingSampler
from detectron2.modeling.proposal_generator.proposal_utils import add_ground_truth_to_proposals
from detectron2.modeling.sampling import subsample_labels
from detectron2.structures import Boxes, Instances, pairwise_iou
from detectron2.modeling.matcher import Matcher
from detectron2.modeling.postprocessing import detector_postprocess
from detectron2.utils.visualizer import Visualizer

from fsdet.modeling.roi_heads.fast_rcnn import FastRCNNOutputs
from fsdet.evaluation import (
    COCOEvaluator, DatasetEvaluators, LVISEvaluator, PascalVOCDetectionEvaluator, verify_results)
from fsdet.evaluation import (
    DatasetEvaluator,
    inference_on_dataset,
    print_csv_format,
    verify_results,
)
import cv2
import torch, torchvision
import logging
import os
from sklearn import svm
from joblib import dump, load
from detectron2.layers import nonzero_tuple
import numpy as np

logging.basicConfig(level=logging.INFO)

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



# Load pretrained detection model and Prediect directly

## Load models of DNN and SVM

In [2]:
cfg = get_cfg()
cfg.merge_from_file('configs/test_unfreeze_lastfews.yaml')
model = build_model(cfg)  # returns a torch.nn.Module
print(cfg.INPUT.FORMAT)

model.eval()
metadata = MetadataCatalog.get(cfg.DATASETS.TEST[0])

# ckpt_file = 'checkpoints/coco/base_model/model_final.pth'
ckpt_file = 'checkpoints/coco/faster_rcnn/30shot_person_unfreeze_lastfews/model_final.pth'
# ckpt_file = 'checkpoints/coco/faster_rcnn/30shot_person_unfreeze_whole/model_0015999.pth'
DetectionCheckpointer(model).load(ckpt_file)

clf = load('svm_results/svm_model_finetuned_prop_base.joblib') 
# clf = load('svm_results/svm_model_finetuned_prop_whole.joblib') 

INFO:fvcore.common.checkpoint:Loading checkpoint from checkpoints/coco/faster_rcnn/30shot_person_unfreeze_lastfews/model_final.pth


BGR


INFO:fvcore.common.checkpoint:Some model parameters or buffers are not found in the checkpoint:
  roi_heads.box_predictor.cls_score.bias


## Register dataset

In [3]:
from detectron2.data.datasets import register_coco_instances
json_dir = 'datasets/coco_experiments/seed1/full_box_30shot_person_trainval.json'
image_dir = 'datasets/coco/trainval2014'
register_coco_instances("30shot_person_train", {}, json_dir, image_dir)

json_dir = 'datasets/coco_experiments/seed1/full_box_1000shot_person_test.json'
image_dir = 'datasets/coco/trainval2014'
register_coco_instances("1000shot_person_test", {}, json_dir, image_dir)


## Difference between some dataloader

In [4]:
# data_loader_train = build_detection_train_loader(cfg)
# data_loader_train_it = iter(data_loader_train)
# data = next(data_loader_train_it)
# print(len(data))
# print(data[0]['image'].shape)
# # print(data)

In [5]:
data_loader_test = build_detection_test_loader(cfg, "1000shot_person_test")
data_loader_test_it = iter(data_loader_test)


INFO:detectron2.data.datasets.coco:Loaded 239 images in COCO format from datasets/coco_experiments/seed1/full_box_1000shot_person_test.json
INFO:detectron2.data.build:Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|   person   | 1000         |
|            |              |
INFO:detectron2.data.dataset_mapper:[DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
INFO:detectron2.data.common:Serializing 239 elements to byte tensors and concatenating them all ...
INFO:detectron2.data.common:Serialized dataset takes 0.67 MiB


In [6]:
data = next(data_loader_test_it)
print(len(data))
print(data[0]['image'].shape)
# print(data)

1
torch.Size([3, 800, 1166])


In [7]:
# transform_gen = T.ResizeShortestEdge(
#             [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST],
#             cfg.INPUT.MAX_SIZE_TEST,
#             )
# print([cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST],
#             cfg.INPUT.MAX_SIZE_TEST)
# data_loader = build_detection_train_loader(DatasetCatalog.get("1000shot_person_test"),
#                                            mapper=DatasetMapper(cfg, is_train=False, augmentations=[transform_gen]),
#                                            total_batch_size = 10)
# data_loader_it = iter(data_loader)
# data = next(data_loader_it)
# print(len(data))
# print(data[0]['image'].shape)
# print(data)

## the type of dataloader is different 'AspectRatio Grouped Dataset' (don't know the reason) 

In [8]:
print(type(data_loader_test))
print(len(data_loader_test))

torch.utils.data.dataloader.DataLoader
239


In [9]:
# run one iter
evaluator = COCOEvaluator("1000shot_person_test", cfg, True, output_dir = cfg.OUTPUT_DIR)
with torch.no_grad():
    inputs = data
    outputs = model(inputs)
    evaluator.reset()
    evaluator.process(inputs, outputs)
    results = evaluator.evaluate()


/home/cloudlet/anaconda3/envs/fsdet/lib/python3.6/site-packages/detectron2/modeling/roi_heads/fast_rcnn.py:124: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370116979/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  filter_inds = filter_mask.nonzero()
INFO:fsdet.evaluation.coco_evaluation:Preparing results for COCO format ...
INFO:fsdet.evaluation.coco_evaluation:Saving results to checkpoints/coco/faster_rcnn/test_0215/coco_instances_results.json
INFO:fsdet.evaluation.coco_evaluation:Evaluating predictions ...
INFO:fsdet.evaluation.coco_evaluation:Evaluation results for bbox: 
|  AP   |  AP50  |  AP75  |  APs  |  APm  |  APl  |
|:-----:|:------:|:------:|:-----:|:-----:|:-----:|
| 0.184 | 0.424  | 0.000  | 0.132 | 0.173 | 0.248 |


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.07s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.004
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.002
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.001
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.002
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=10

In [10]:
print(outputs[0])

{'instances': Instances(num_instances=100, image_height=343, image_width=500, fields=[pred_boxes: Boxes(tensor([[307.4873, 139.3737, 430.1411, 278.2307],
        [ 34.8617, 131.7565,  53.4179, 167.0400],
        [264.2541, 110.1587, 447.5597, 305.4304],
        [228.1001, 113.6795, 278.5739, 158.8312],
        [219.2898, 125.5122, 239.9398, 156.1071],
        [ 60.2247, 118.4398,  77.9698, 158.3530],
        [439.1581, 112.6526, 492.3134, 159.4070],
        [245.5859, 112.5854, 263.1079, 165.0826],
        [229.1883, 114.7197, 248.3466, 159.9225],
        [261.0497, 115.3187, 275.6426, 161.9311],
        [137.4505, 124.4982, 237.0562, 156.1336],
        [183.7613, 115.2427, 215.6950, 196.0591],
        [ 35.8171, 118.0838, 309.2529, 270.4918],
        [ 11.0887, 137.1735,  34.2363, 157.8231],
        [106.0640,  98.7603, 430.5492, 266.2970],
        [ 29.0133, 124.1063, 161.5993, 152.6946],
        [170.5462, 117.1802, 230.6380, 171.0150],
        [251.7086, 114.5926, 270.1111, 163.988

In [11]:
# default test 
inference_on_dataset(model, data_loader_test, evaluator)

INFO:fsdet.evaluation.evaluator:Start inference on 239 images
INFO:fsdet.evaluation.evaluator:Inference done 50/239. 0.0523 s / img. ETA=0:00:09
INFO:fsdet.evaluation.evaluator:Inference done 100/239. 0.0529 s / img. ETA=0:00:07
INFO:fsdet.evaluation.evaluator:Inference done 150/239. 0.0530 s / img. ETA=0:00:04
INFO:fsdet.evaluation.evaluator:Inference done 200/239. 0.0530 s / img. ETA=0:00:02
INFO:fsdet.evaluation.evaluator:Total inference time: 0:00:12 (0.051282 s / img per device, on 1 devices)
INFO:fsdet.evaluation.evaluator:Total inference pure compute time: 0:00:12 (0.051817 s / img per device, on 1 devices)
INFO:fsdet.evaluation.coco_evaluation:Preparing results for COCO format ...
INFO:fsdet.evaluation.coco_evaluation:Saving results to checkpoints/coco/faster_rcnn/test_0215/coco_instances_results.json
INFO:fsdet.evaluation.coco_evaluation:Evaluating predictions ...


Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


INFO:fsdet.evaluation.coco_evaluation:Evaluation results for bbox: 
|  AP   |  AP50  |  AP75  |  APs  |  APm  |  APl  |
|:-----:|:------:|:------:|:-----:|:-----:|:-----:|
| 3.497 | 11.452 | 0.857  | 2.916 | 2.782 | 6.296 |


DONE (t=1.14s).
Accumulating evaluation results...
DONE (t=0.07s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.115
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.009
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.029
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.028
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.063
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.041
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.123
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.167
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.092
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.157
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= lar

OrderedDict([('bbox',
              {'AP': 3.4974809343450457,
               'AP50': 11.452398492432149,
               'AP75': 0.8571714524642489,
               'APs': 2.9159147665604803,
               'APm': 2.781923164794646,
               'APl': 6.295968797067547})])

# Run detector step by step (should have same results)

In [12]:
print(len(data))
print(data[0]['image'].shape)
# print(data)

1
torch.Size([3, 800, 1166])


In [13]:
assert len(cfg.MODEL.PIXEL_MEAN) == len(cfg.MODEL.PIXEL_STD)
num_channels = len(cfg.MODEL.PIXEL_MEAN)
device = 'cuda'
pixel_mean = (
    torch.Tensor(cfg.MODEL.PIXEL_MEAN)
    .to(device)
    .view(num_channels, 1, 1)
)
pixel_std = (
    torch.Tensor(cfg.MODEL.PIXEL_STD)
    .to(device)
    .view(num_channels, 1, 1)
)
normalizer = lambda x: (x - pixel_mean) / pixel_std

In [14]:
evaluator = COCOEvaluator("1000shot_person_test", cfg, True, output_dir = cfg.OUTPUT_DIR)
evaluator.reset()

training = False
save_results = True
box2box_transform = Box2BoxTransform(
            weights=cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_WEIGHTS
        )
smooth_l1_beta = cfg.MODEL.ROI_BOX_HEAD.SMOOTH_L1_BETA
test_score_thresh        = cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST
# test_score_thresh        = 0.7
test_nms_thresh          = cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST
test_detections_per_img  = cfg.TEST.DETECTIONS_PER_IMAGE


with torch.no_grad():
    for idx, inputs in enumerate(data_loader_test):
        #     batched_inputs = data
        batched_inputs = inputs
        ###################################    
        #     outputs = model(inputs)     #
        #---------------------------------# 

        # Normalize, pad and batch the input images. (Preprocess_image)
        images = [x["image"].to('cuda') for x in batched_inputs]
        images = [normalizer(x) for x in images]
        images = ImageList.from_tensors(
            images, model.backbone.size_divisibility
        )

        # forward
        features = model.backbone(images.tensor)
#         print('features shape:', features['p3'].shape)
        proposals, _ = model.proposal_generator(images, features)
#         print('proposal num per img:', proposals_list[0].objectness_logits.shape)


        #     results, _ = model.roi_heads(images, features, proposals)
        #     print('\ninstance for image 0:', results[0], '\n')

        # run roi_heads step by step
        if training:
        #         proposals = [proposal for proposal in proposals]
            targets = [d['instances'].to('cuda') for d in data]
            proposals = model.roi_heads.label_and_sample_proposals(proposals, targets)

        box_features = model.roi_heads.box_pooler(
            [features[f] for f in ["p2", "p3", "p4", "p5"]], [x.proposal_boxes for x in proposals]
        )
#         print(box_features.shape)
        box_features = model.roi_heads.box_head(box_features)
#         print(box_features.shape)

        pred_class_logits, pred_proposal_deltas = model.roi_heads.box_predictor(
            box_features
        )
#         print('pred_class_logits', pred_class_logits[:3])
#         print('pred_proposal_deltas', pred_proposal_deltas.shape)

        outputs = FastRCNNOutputs(
            box2box_transform,
            pred_class_logits,
            pred_proposal_deltas,
            proposals,
            smooth_l1_beta,
        )

        results, _ = outputs.inference(
                test_score_thresh,
                test_nms_thresh,
                test_detections_per_img,
        )

        # postprocess: resize images
        processed_results = []
        for results_per_image, input_per_image, image_size in zip(
            results, batched_inputs, images.image_sizes
        ):
            height = input_per_image.get("height", image_size[0])
            width = input_per_image.get("width", image_size[1])
            r = detector_postprocess(results_per_image, height, width)
            processed_results.append({"instances": r})
#         print('postprocessed instance for image 0:\n', processed_results[0], '\n')
###################################  

        # evaluate
        evaluator.process(inputs, processed_results)
        # visualizer
        # inputs should be only one image
        raw_image = cv2.imread(batched_inputs[0]['file_name'])
        result_show = processed_results[0]["instances"]
        v = Visualizer(raw_image,
                        metadata=MetadataCatalog.get("1000shot_person_test"), 
                        scale=1.0, 
                        instance_mode=ColorMode.IMAGE   # remove the colors of unsegmented pixels
            )
        v = v.draw_instance_predictions(result_show.to("cpu"))
        if save_results:
            folder_name = './test_output/det/'
            os.makedirs(folder_name, exist_ok=True)
            det_img_dir = folder_name + str(idx) + '.jpg'
            cv2.imwrite(det_img_dir, v.get_image())
    eval_results = evaluator.evaluate()

INFO:fsdet.evaluation.coco_evaluation:Preparing results for COCO format ...
INFO:fsdet.evaluation.coco_evaluation:Saving results to checkpoints/coco/faster_rcnn/test_0215/coco_instances_results.json
INFO:fsdet.evaluation.coco_evaluation:Evaluating predictions ...


Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


INFO:fsdet.evaluation.coco_evaluation:Evaluation results for bbox: 
|  AP   |  AP50  |  AP75  |  APs  |  APm  |  APl  |
|:-----:|:------:|:------:|:-----:|:-----:|:-----:|
| 3.497 | 11.452 | 0.857  | 2.916 | 2.782 | 6.296 |


DONE (t=1.24s).
Accumulating evaluation results...
DONE (t=0.07s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.115
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.009
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.029
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.028
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.063
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.041
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.123
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.167
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.092
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.157
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= lar

# Add SVM classifier as the final layer

In [15]:
evaluator = COCOEvaluator("1000shot_person_test", cfg, True, output_dir = cfg.OUTPUT_DIR)
evaluator.reset()
training = False
box2box_transform = Box2BoxTransform(
            weights=cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_WEIGHTS
        )
smooth_l1_beta = cfg.MODEL.ROI_BOX_HEAD.SMOOTH_L1_BETA
# test_score_thresh        = cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST
test_score_thresh        = 0.5
test_nms_thresh          = cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST
test_detections_per_img  = cfg.TEST.DETECTIONS_PER_IMAGE
print(test_score_thresh, test_nms_thresh, test_detections_per_img)

with torch.no_grad():
    for idx, inputs in enumerate(data_loader_test):
        #     batched_inputs = data
        batched_inputs = inputs
    ###################################    
    #     outputs = model(inputs)     #
    #---------------------------------# 

        # Normalize, pad and batch the input images. (Preprocess_image)
        images = [x["image"].to('cuda') for x in batched_inputs]
        images = [normalizer(x) for x in images]
        images = ImageList.from_tensors(
            images, model.backbone.size_divisibility
        )

        # forward
        features = model.backbone(images.tensor)
#         print('features shape:', features['p3'].shape)
        proposals, _ = model.proposal_generator(images, features)
#         print('proposal num per img:', proposals_list[0].objectness_logits.shape)


    #     results, _ = model.roi_heads(images, features, proposals)
    #     print('\ninstance for image 0:', results[0], '\n')

        # run roi_heads step by step
        if training:
    #         proposals = [proposal for proposal in proposals]
            targets = [d['instances'].to('cuda') for d in data]
            proposals = model.roi_heads.label_and_sample_proposals(proposals, targets)

        box_features = model.roi_heads.box_pooler(
            [features[f] for f in ["p2", "p3", "p4", "p5"]], [x.proposal_boxes for x in proposals]
        )
#         print(box_features.shape)
        box_features = model.roi_heads.box_head(box_features)
#         print(box_features.shape)

        pred_class_logits, pred_proposal_deltas = model.roi_heads.box_predictor(
            box_features
        )
#         print('pred_class_logits', pred_class_logits[:3])
#         print('pred_proposal_deltas', pred_proposal_deltas.shape)

        outputs = FastRCNNOutputs(
            box2box_transform,
            pred_class_logits,
            pred_proposal_deltas,
            proposals,
            smooth_l1_beta,
        )

        results, _ = outputs.inference(
                test_score_thresh,
                test_nms_thresh,
                test_detections_per_img,
        )

        # postprocess: resize images
        processed_results = []
        for results_per_image, input_per_image, image_size in zip(
            results, batched_inputs, images.image_sizes
        ):
            height = input_per_image.get("height", image_size[0])
            width = input_per_image.get("width", image_size[1])
            r = detector_postprocess(results_per_image, height, width)
            processed_results.append({"instances": r})
#         print('postprocessed instance for image 0:\n', processed_results[0], '\n')

        # SVM 
        X = box_features.to('cpu').detach().numpy()
#         y_hat = clf.predict(X)
        pred_class_logits_svm = clf.predict_log_proba(X)
#         pred_class_logits_svm = np.zeros((pred_class_logits.shape))
#         for i in range(y_hat.shape[0]):
#             # if y_hat=1, background; if y_hat=0, person
#             pred_class_logits_svm[i][0] = 1 - y_hat[i]
#             pred_class_logits_svm[i][-1] = y_hat[i]
        pred_class_logits_svm = torch.from_numpy(pred_class_logits_svm).to('cuda')
#         print(y_hat.shape)
#         print(pred_class_logits_svm[:3])

        outputs_svm = FastRCNNOutputs(
            box2box_transform,
            pred_class_logits_svm,
            pred_proposal_deltas,
            proposals,
            smooth_l1_beta,
        )

        pred_instances_svm, _ = outputs_svm.inference(
                test_score_thresh,
                test_nms_thresh,
                test_detections_per_img,
        )

        processed_results_svm = []
        for results_per_image, input_per_image, image_size in zip(
            pred_instances_svm, batched_inputs, images.image_sizes
        ):
            height = input_per_image.get("height", image_size[0])
            width = input_per_image.get("width", image_size[1])
            r = detector_postprocess(results_per_image, height, width)
            processed_results_svm.append({"instances": r})
#         print('\n\nSVM postprocessed instance for image 0:\n', processed_results_svm[0], '\n')  
    
###################################  
        # evaluate
        evaluator.process(inputs, processed_results_svm)
        # visualizer
        # inputs should be only one image
        raw_image = cv2.imread(batched_inputs[0]['file_name'])
        result_show = processed_results_svm[0]["instances"]
        v = Visualizer(raw_image,
                        metadata=MetadataCatalog.get("1000shot_person_test"), 
                        scale=1.0, 
                        instance_mode=ColorMode.IMAGE   # remove the colors of unsegmented pixels
            )
        v = v.draw_instance_predictions(result_show.to("cpu"))
        if save_results:
#             det_img_folder = time.strftime("%d_%H_%M/")
            folder_name = './test_output/svm_prob/'
            os.makedirs(folder_name, exist_ok=True)
            det_img_dir = folder_name + str(idx) + '.jpg'
            cv2.imwrite(det_img_dir, v.get_image())
    results = evaluator.evaluate()

0.5 0.5 100


INFO:fsdet.evaluation.coco_evaluation:Preparing results for COCO format ...
INFO:fsdet.evaluation.coco_evaluation:Saving results to checkpoints/coco/faster_rcnn/test_0215/coco_instances_results.json
INFO:fsdet.evaluation.coco_evaluation:Evaluating predictions ...


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


INFO:fsdet.evaluation.coco_evaluation:Evaluation results for bbox: 
|  AP   |  AP50  |  AP75  |  APs  |  APm  |  APl   |
|:-----:|:------:|:------:|:-----:|:-----:|:------:|
| 4.675 | 14.536 | 1.162  | 3.389 | 5.262 | 11.502 |


DONE (t=0.64s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.047
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.145
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.012
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.053
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.115
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.047
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.129
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.156
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.072
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.138
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= lar